In [2]:
from ultralytics import YOLO
import cv2

# Cargar el modelo preentrenado (por ejemplo, YOLOv8n, YOLOv8s, etc.)
model = YOLO("yolov8n.pt") 

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/dani/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:01<00:00, 5.57MB/s]


In [3]:
# Cargar imagen
image_path = "example.jpg"
results = model(image_path)
print(results)  # Imprimir resultados


image 1/1 /home/dani/Desktop/futbol/example.jpg: 416x640 5 persons, 30.6ms
Speed: 2.7ms preprocess, 30.6ms inference, 79.9ms postprocess per image at shape (1, 3, 416, 640)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banan

In [4]:
# Filtrar personas y mostrar resultados
for result in results:
    boxes = result.boxes
    for box in boxes:
        cls_id = int(box.cls[0])  # Clase detectada
        if model.names[cls_id] == 'person':
            x1, y1, x2, y2 = box.xyxy[0].tolist()  # Coordenadas de la caja
            print(f"Persona detectada en: ({x1}, {y1}), ({x2}, {y2})")

Persona detectada en: (112.66673278808594, 18.499736785888672), (423.9985656738281, 377.3861083984375)
Persona detectada en: (34.229591369628906, 84.89447021484375), (108.04861450195312, 295.09075927734375)
Persona detectada en: (537.9708251953125, 125.15662384033203), (592.1982421875, 233.28688049316406)
Persona detectada en: (371.10302734375, 67.37295532226562), (427.9504089355469, 314.15460205078125)
Persona detectada en: (117.51016998291016, 122.64320373535156), (219.3135528564453, 310.6005554199219)


In [5]:
# Mostrar la imagen con las detecciones
results[0].plot()  # Dibuja las cajas
cv2.imshow("Detecciones", results[0].plot())
cv2.waitKey(0)
cv2.destroyAllWindows()

: 

: 

# Video

In [17]:
from ultralytics import YOLO
import cv2
import torch
import numpy as np

print("GPU disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Usando:", torch.cuda.get_device_name(0))

model = YOLO("yolov8n.pt")
model.to('cuda')  
model.to('cuda' if torch.cuda.is_available() else 'cpu')
print("Modelo cargado en:", model.device)

GPU disponible: True
Usando: NVIDIA GeForce RTX 4060
Modelo cargado en: cuda:0


In [ ]:
cap = cv2.VideoCapture("SoccerNet/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_720p.mkv")


if not cap.isOpened():
    print("No se pudo abrir el video.")
    exit()
# Rango para el color blanco
lower_white = np.array([30, 10, 210])
upper_white = np.array([110, 70, 255])
prev_frame = None
while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model.predict(frame, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False)
    annotated_frame = frame.copy()
    for result in results:
        boxes = result.boxes
        for box in boxes:
            cls_id = int(box.cls[0])  # Clase detectada
            if model.names[cls_id] == 'person':
                x1, y1, x2, y2 = box.xyxy[0].tolist()  # Coordenadas de la caja
                #nos quedamos con los enteros
                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
    #convertimos a HSV
    hsv_frame = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2HSV)
    # Crear una máscara para el color blanco
    mask = cv2.inRange(hsv_frame, lower_white, upper_white)
    # Aplicar la máscara a la imagen original
    masked_frame = cv2.bitwise_and(annotated_frame, annotated_frame, mask=mask)
    # Aplicar detección de bordes
    edges = cv2.Canny(masked_frame, 50, 150)
    # Detección de líneas
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=100, maxLineGap=10)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(annotated_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
    #Mostrar mascara
    cv2.imshow("Mascara", masked_frame)
    cv2.imshow("Bordes del Video", edges)
    # Mostrar la imagen con las detecciones
    cv2.imshow("Detecciones", annotated_frame)
    # Esperar hasta que se presione una tecla
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 